In [2]:
import cv2
import numpy as np
import math
cap = cv2.VideoCapture(0)
i=0   
li=[]
for i in range(0,4):
    while(1):
       
        try:  #an error comes if it does not find anything in window as it cannot find contour of max area
              #therefore this try error statement

            ret, frame = cap.read()
            ret, frame1 = cap.read()
            frame=cv2.flip(frame,1)
            frame1=cv2.flip(frame1,1)
            
            kernel = np.ones((3,3),np.uint8)

            #define region of interest
            roi=frame[25:350, 25:350]
            roi1=frame1[350:650, 50:350]
        


            cv2.rectangle(frame,(25,25),(350,350),(0,255,0),0)    
            hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            cv2.rectangle(frame1,(350,50),(650,350),(0,255,0),0)    
            hsv1 = cv2.cvtColor(roi1, cv2.COLOR_BGR2HSV)



        # define range of skin color in HSV
            lower_skin = np.array([0,20,70], dtype=np.uint8)
            upper_skin = np.array([20,255,255], dtype=np.uint8)
            lower_skin1= np.array([0,20,70], dtype=np.uint8)
            upper_skin1= np.array([20,255,255], dtype=np.uint8)

         #extract skin colur imagw  
            mask = cv2.inRange(hsv, lower_skin, upper_skin)
            mask1 = cv2.inRange(hsv1, lower_skin1, upper_skin1)



        #extrapolate the hand to fill dark spots within
            mask = cv2.dilate(mask,kernel,iterations = 4)
            mask1 = cv2.dilate(mask1,kernel,iterations = 4)

        #blur the image
            mask = cv2.GaussianBlur(mask,(5,5),100)
            mask1 = cv2.GaussianBlur(mask1,(5,5),100)



        #find contours
            _,contours,hierarchy= cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            _,contours1,hierarchy1= cv2.findContours(mask1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

       #find contour of max area(hand)
            cnt = max(contours, key = lambda x: cv2.contourArea(x))
            cnt1 = max(contours1, key = lambda x: cv2.contourArea(x))

        #approx the contour a little
            epsilon = 0.0005*cv2.arcLength(cnt,True)
            epsilon1 = 0.0005*cv2.arcLength(cnt1,True)

            approx= cv2.approxPolyDP(cnt,epsilon,True)
            approx1= cv2.approxPolyDP(cnt1,epsilon1,True)


        #make convex hull around hand
            hull = cv2.convexHull(cnt)
            hull1 = cv2.convexHull(cnt1)

         #define area of hull and area of hand
            areahull = cv2.contourArea(hull)
            areahull1 = cv2.contourArea(hull1)
            areacnt = cv2.contourArea(cnt)
            areacnt1 = cv2.contourArea(cnt1)

        #find the percentage of area not covered by hand in convex hull
            arearatio=((areahull-areacnt)/areacnt)*100
            arearatio1=((areahull1-areacnt1)/areacnt1)*100

         #find the defects in convex hull with respect to hand
            hull = cv2.convexHull(approx, returnPoints=False)
            hull1 = cv2.convexHull(approx1, returnPoints=False)
            defects = cv2.convexityDefects(approx, hull)
            defects1 = cv2.convexityDefects(approx1, hull1)

        # l = no. of defects
            l=0
            l1=0

        #code for finding no. of defects due to fingers
            for i in range(defects.shape[0]):
                s,e,f,d = defects[i,0]
                start = tuple(approx[s][0])
                end = tuple(approx[e][0])
                far = tuple(approx[f][0])
                pt= (100,180)


                # find length of all sides of triangle
                a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                s = (a+b+c)/2
                ar = math.sqrt(s*(s-a)*(s-b)*(s-c))

                #distance between point and convex hull
                d=(2*ar)/a

                # apply cosine rule here
                angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57


                # ignore angles > 90 and ignore points very close to convex hull(they generally come due to noise)
                if angle <= 90 and d>30:
                    l += 1
                    cv2.circle(roi, far, 3, [255,0,0], -1)

                #draw lines around hand
                cv2.line(roi,start, end, [0,255,0], 2)
            
            l+=1
            l1+=1

            #print corresponding gestures which are in their ranges
            font = cv2.FONT_HERSHEY_SIMPLEX
            if l==1:
                if areacnt<2000:
                    cv2.putText(frame,'Put hand in the box',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                else:
                    if arearatio<12:
                        cv2.putText(frame,'0',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    elif arearatio<17.5:
                        cv2.putText(frame,'Best of luck',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

                    else:
                        cv2.putText(frame,'1',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            elif l==2:
                cv2.putText(frame,'2',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            elif l==3:
                cv2.putText(frame,'3',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            elif l==4:
                cv2.putText(frame,'4',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            elif l==5:
                cv2.putText(frame,'5',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            elif l==6:
                cv2.putText(frame,'reposition',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            else :
                cv2.putText(frame,'reposition',(10,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

            
            
            #show the windows
            cv2.imshow('mask',mask)
            cv2.imshow('frame',frame)
            cv2.imshow('frame1',frame1)
            cv2.imshow('mask',mask1)
            
        except:
            pass


        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break
        
    print(l)
    li.append(l)
    
cv2.destroyAllWindows()
cap.release()  



ModuleNotFoundError: No module named 'cv2'

In [3]:
!pip install cv2

ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2
